## Neural Network Regression

Neural Networks are first introduced with image recognition but in general neural networks are used for clustering through unsupervised learning, classification and regression through supervised learning. It not only helps to group unlabeled data, categorize labeled data but also predict continuous values.

For neural network regression, we have to scale the values to a range of 0 to 1. It is similar to Standard scalar with a specified range in Machine Learning.

In classification model we swap the softmax layer at the end with a linear layer, and swap the cross-entropy loss with root mean squared error or mean squared error.<br><br>

### Initialization factors:

There are a number of important, and sometimes subtle, choices that need to be made when building and training a neural network. You have to decide how many layers to have, which optimization algorithm is best suited for the network, etc. The initial weight plays a very important role on convergence rate and the quality of the network. <br><br>

### Major types of normalization: 

1. ReLU: ReLU Layer applies an elementwise activation function max(0,x), which turns negative values to zeros (thresholding at zero). This layer does not change the size of the volume and there are no hyperparameters.

2. Tanh: The range of the tanh function is [-1,1]

3. Sigmoid: The range of sigmoid function is [0,1]<br><br>


### Regularization:

Dropout forces an artificial neural network to learn multiple independent representations of the same data by alternately randomly disabling neurons in the learning phase. Dropout is a vital feature in almost every state-of-the-art neural network implementation. To perform dropout on a layer, you randomly set some of the layer's values to 0 during forward propagation.<br><br>

In [1]:
#import the libraries
import pandas as pd
from sklearn.cross_validation import train_test_split
import tensorflow as tf
import numpy as np
import warnings 
from sklearn.preprocessing import MinMaxScaler

C:\Users\Baby Panda\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Baby Panda\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#importing the data 
data = pd.read_csv('modified_train.csv')

In [3]:
#Selecting the features for neural network regression, here the predictors are all the selected variables according to the hypothesis
data = data.drop(['Unnamed: 0'], axis=1)
#Selecting Target variable, which is saleprice
y = data['SalePrice'].as_matrix()
del data['SalePrice']


#selecting the predictors, all the variables except saleprice
X = data.as_matrix()

#Dividing the data to test and train for regression
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)


For neural network regression, we have to scale the values to a range of 0 to 1. It is similar to Standard scalar with a specified range in Machine Learning.Performing a scaling on test set is equally important

In [4]:
#reshaping y variables
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

# Like standard scalar for simple ML algorithms, in Neural networks we have to convert the values to a range from 0 to 1
X_scaler = MinMaxScaler(feature_range=(0, 1))
Y_scaler = MinMaxScaler(feature_range=(0, 1))

# Scaling the training variables
X_scaled_training = X_scaler.fit_transform(X_train)
Y_scaled_training = Y_scaler.fit_transform(y_train)

# Scaling the test variables
X_scaled_testing = X_scaler.transform(X_test)
Y_scaled_testing = Y_scaler.transform(y_test)


In [12]:
# Define model parameters
learning_rate = 0.003
training_epochs = 1500
batch_size = 10
dropout_rate = 0.7
total_len = X_train.shape[0]
input_nodes = X_train.shape[1]
output_nodes = 1


# Define how many neurons we want in each layer of our neural network
l1_nodes = 150
l2_nodes = 250
l3_nodes = 300
l4_nodes = 450

In [13]:
# define placeholders
x = tf.placeholder("float", [None, 53])
y = tf.placeholder("float", [None, 1])


# Create model
def model(x, weights, biases):
    with tf.name_scope('neural_network'):
        layer_1 = tf.add(tf.matmul(x, weights['weight1']), biases['biase1'])
        layer_1 = tf.nn.relu(layer_1)
        layer_2 = tf.add(tf.matmul(layer_1, weights['weight2']), biases['biase2'])
        layer_2 = tf.nn.relu(layer_2)
        layer_3 = tf.add(tf.matmul(layer_2, weights['weight3']), biases['biase3'])
        layer_3 = tf.nn.relu(layer_3)
        layer_4 = tf.add(tf.matmul(layer_3, weights['weight4']), biases['biase4'])
        layer_4 = tf.nn.relu(layer_4)
        out_layer = tf.matmul(layer_4, weights['out']) + biases['out']
        return out_layer

# weights and bias for each layer
with tf.name_scope('weights_biases'):
    weights = {
        'weight1': tf.Variable(tf.random_normal([input_nodes, l1_nodes], 0, 0.17)),
        'weight2': tf.Variable(tf.random_normal([l1_nodes, l2_nodes], 0, 0.007)),
        'weight3': tf.Variable(tf.random_normal([l2_nodes, l3_nodes], 0, 0.007)),
        'weight4': tf.Variable(tf.random_normal([l3_nodes, l4_nodes], 0, 0.007)),
        'out': tf.Variable(tf.random_normal([l4_nodes, output_nodes], 0, 0.007))
    }
    biases = {
        'biase1': tf.Variable(tf.random_normal([l1_nodes], 0, 0.03)),
        'biase2': tf.Variable(tf.random_normal([l2_nodes], 0, 0.03)),
        'biase3': tf.Variable(tf.random_normal([l3_nodes], 0, 0.003)),
        'biase4': tf.Variable(tf.random_normal([l4_nodes], 0, 0.03)),
        'out': tf.Variable(tf.random_normal([output_nodes], 0, 0.03))
    }

# Model definition
with tf.name_scope("train"):
    y_pred = model(x, weights, biases)
    
    #cost calculates the rmse value
    cost = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(y_pred, y))))
    
    #Adam optimizer to reduce the rmse value
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # Train dataset
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(total_len/batch_size)
        # Loop over all batches
        for i in range(total_batch-1):
            batch_x = X_train[i*batch_size:(i+1)*batch_size]
            batch_y = y_train[i*batch_size:(i+1)*batch_size]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c, p = sess.run([optimizer, cost, y_pred], feed_dict={x: batch_x,y: batch_y})
                                                          
            # Compute average loss
            avg_cost += c / total_batch

        # sample prediction
        label_value = batch_y
        estimate = p
        err = label_value-estimate
        #print ("num batch:", total_batch)

        # Display logs per epoch step
        if epoch % 100 == 0:
            print ("Epoch:", '%04d' % (epoch+1), "cost=", "{:.4f}".format(avg_cost))
            print ("----------------------------")
           

    # Test model
    accuracy = sess.run(cost, feed_dict={x: X_test, y: y_test})
    predicted_vals = sess.run(y_pred, feed_dict={x: X_test})
    print ("----------------------------")
    print ("Accuracy:", accuracy)
    

Epoch: 0001 cost= 70337.1364
----------------------------
Epoch: 0101 cost= 30001.4291
----------------------------
Epoch: 0201 cost= 27310.2516
----------------------------
Epoch: 0301 cost= 26125.4072
----------------------------
Epoch: 0401 cost= 26856.8483
----------------------------
Epoch: 0501 cost= 25141.2505
----------------------------
Epoch: 0601 cost= 23340.3170
----------------------------
Epoch: 0701 cost= 21846.9591
----------------------------
Epoch: 0801 cost= 22470.4396
----------------------------
Epoch: 0901 cost= 21747.8114
----------------------------
Epoch: 1001 cost= 21956.1568
----------------------------
Epoch: 1101 cost= 21446.6551
----------------------------
Epoch: 1201 cost= 21406.0626
----------------------------
Epoch: 1301 cost= 23782.5771
----------------------------
Epoch: 1401 cost= 20922.3198
----------------------------
----------------------------
Accuracy: 35735.082


### Link to [Kaggle kernel](https://github.com/hmangrola/Predicting-House-Prices-Ames-Iowa/blob/master/Kaggle_Kernel.ipynb) notebook

This kernel is licensed (license can be found on github repository)